# Bayesian VOO

In [ ]:
import ray
import numpy as np
import matplotlib.pyplot as plt
from util import cos_exp_square_nd,run_bavoo,plot_line
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
np.set_printoptions(precision=3)
print ("Packages Loaded.")

In [ ]:
def func_eval(x): # x:[n x d]
    y = cos_exp_square_nd(x)
    return y # y:[n x 1]

### Run BAVOO, VOO, and BO

In [ ]:
xmin,xmax,seed,VERBOSE = -2.0,+2.0,0,True
USE_RAY,n_worker,n_sample_for_bo,max_try_sbv = True,10,5000,20000
ray.init(configure_logging=False)
for xdim in [2,3,4,5]:
    n_data_max = 200*int(1.5**xdim)
    x_minmax = np.tile(np.array([[xmin,xmax]]),(xdim,1))
    # Run BAVOO
    x_data_bv,y_data_bv = run_bavoo(func_eval,x_minmax,USE_RAY=USE_RAY,
        n_random=1,n_bo=1,n_voo=1,n_cd=1,n_data_max=n_data_max,n_worker=n_worker,seed=seed,
        n_sample_for_bo=n_sample_for_bo,max_try_sbv=max_try_sbv,save_folder='',VERBOSE=VERBOSE)
    # Run VOO
    x_data_voo,y_data_voo = run_bavoo(func_eval,x_minmax,USE_RAY=USE_RAY,
        n_random=1,n_bo=0,n_voo=1,n_cd=0,n_data_max=n_data_max,n_worker=n_worker,seed=seed,
        n_sample_for_bo=n_sample_for_bo,max_try_sbv=max_try_sbv,save_folder='',VERBOSE=VERBOSE)
    # Run BO
    x_data_bo,y_data_bo = run_bavoo(func_eval,x_minmax,USE_RAY=USE_RAY,
        n_random=0,n_bo=1,n_voo=0,n_cd=0,n_data_max=n_data_max,n_worker=n_worker,seed=seed,
        n_sample_for_bo=n_sample_for_bo,max_try_sbv=max_try_sbv,save_folder='',VERBOSE=VERBOSE)
    # Plot results
    xin = np.linspace(0,n_data_max-1,n_data_max).reshape((-1,1))
    plot_line(x=xin,y=np.minimum.accumulate(y_data_bv[:n_data_max,0:1]),fmt='-',lc='r',label='BAVOO',
              x2=xin,y2=np.minimum.accumulate(y_data_bo[:n_data_max,0:1]),fmt2=':',lc2='b',label2='BO',
              x3=xin,y3=np.minimum.accumulate(y_data_voo[:n_data_max,0:1]),fmt3='--',lc3='m',label3='VOO',
              tstr='Input dimension is [%d]'%(xdim),tfs=20,
              lloc='upper right')
ray.shutdown()